In [ ]:
import os
import sys
sys.path.append(os.path.abspath('..'))
from config import *

import random
import numpy as np
import torch
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

# 演習：レントゲン画像歯検出

歯や歯茎の状態を確認するためにレントゲンを使用することは、歯科検査において欠かせません。もし、レントゲン画像から歯の領域を自動で検出し、その状態、たとえば虫歯の有無を判定できるようになれば、歯科医師の負担を軽減し、診断をより迅速かつ正確に行うことが可能です。このような自動化を実現するには、画像内の歯の領域を正確に分離するセグメンテーション技術が必要です。本節では、レントゲン画像を用いた歯のセグメンテーションの方法を学び、歯科検査を効率化する支援方法を考えていきます。

## 演習準備

### ライブラリ

本節で必要なライブラリを読み込みます。os、random、NumPy、Pandas、Matplotlib、Pillow（PIL）は訓練過程の可視化や推論結果の表示に利用します。scikit-image（skimage）はマスクから輪郭線を計算する際に使用します。さらに、torch、torchvision、torchmetrics はインスタンスセグメンテーションモデルの訓練、検証、推論に利用します。

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL

import skimage

import torch
import torchvision
import torchvision.transforms.v2
import torchmetrics

print(f'torch v{torch.__version__}; torchvision v{torchvision.__version__}')

ライブラリの読み込み時に *ImportError* や *ModuleNotFoundError* が発生した場合は、該当するライブラリをインストールしてください。ライブラリのバージョンを揃える必要はありませんが、PyTorch（torch）および torchvision が上記のバージョンと異なる時、実行中に警告メッセージが現れたり、同じ結果にならなかったりする可能性があります。

### データセット

本章では、Kaggle にて CC0 ライセンスのもとで公開されている [Teeth Segmentation on dental X-ray images](https://www.kaggle.com/datasets/humansintheloop/teeth-segmentation-on-dental-x-ray-images) を使用します。このデータセットは、歯のレントゲン画像からなるデータセットであり、歯を領域をポリゴン囲んだアノテーションが含まれています。アノテーションはポリゴンの座標を記載した数値データと画像として保存されたマスクの両方が用意されています。マスク画像は RGB カラー画像で、画像内の色が各歯の番号に対応しています。例えば、13 番目の歯は色が (R, G, B) = (1, 1, 1)、12 番目の歯は (2, 2, 2) のように、32 番目の歯までそれぞれ異なる色で対応付けられています[^teethnumber]。歯の番号と色の対応関係は、`obj_class_to_machine_color.json` ファイルに保存されています。歯の番号を区別して取り扱う際は、この対応表を使用してデータを取得する必要があります。

```{figure} ../_static/teethsegm_dataset.jpg
---
name: teethsegm_dataset_example
---
Teeth Segmentation on dental X-ray images データセットのサンプル画像とマスク画像。
```

オリジナルのデータセットはやや大きいため、本節では、オリジナルのデータセットから 80 枚の画像を抽出して、そのうち 60 枚を訓練データ、10 枚を検証データ、10 枚を検証データとして整理したものを利用します。Jupyter Notebook では、以下のコマンドを実行することで、データセットをダウンロードできます。

```bash
!wget https://dl.biopapyrus.jp/data/teethsegm.zip
!unzip teethsegm.zip
```

[^teethnumber]: アメリカでは、歯の番号は右上から始まり、左上、左下、右下の順に通し番号が付けられます。大臼歯（親知らず）は 1、16、17、32 番にあたります。

### 前処理

本節では、歯の番号を区別せずにインスタンスセグメンテーションを行います。そのために、前述のデータセットをインスタンスセグメンテーションの学習に利用できる形式に変換する処理（`TeethDataset`）を定義します。

Teeth Segmentation on dental X-ray images データセットに含まれるマスク画像は RGB 画像で、例えば画素値が (1, 1, 1) の部分が 13 番目の歯、(2, 2, 2) の部分が 12 番目の歯に対応しています。`TeethDataset` では、この RGB 画像を基に、まず画素値が (1, 1, 1) の部分を判定して 1 枚のバイナリマスクを作成し、次に画素値が (2, 2, 2) の部分について同様にバイナリマスクを作成します。この操作を繰り返し、32 本の歯に対応するマスクを生成します（`(mask == labels[:, None, None]).to(dtype=torch.uint8)`）。さらに、画像内に該当する歯が存在しない場合、その歯に対応するマスクの画素値はすべて 0 になります。このような無効なマスクを削除する処理（`masks[has_tooth]`）も実装しています。最後に、生成したマスクを PyTorch で扱える形式に変換する処理を行います。

In [ ]:
class TeethDataset(torch.utils.data.Dataset):
    def __init__(self, root):
        self.root = root
        self.images, self.masks = self.__load_datasets(self.root)
        self.transforms = torchvision.transforms.v2.Compose([
            torchvision.transforms.v2.ToDtype(torch.float, scale=True),
            torchvision.transforms.v2.ToPureTensor()
        ])
    
    def __getitem__(self, idx):
        image = torchvision.io.read_image(self.images[idx])
        mask = torchvision.io.read_image(self.masks[idx])

        # create labels, masks, bboxes for training from the original mask
        labels = torch.tensor([_ for _ in range(1, 33)])
        masks = (mask == labels[:, None, None]).to(dtype=torch.uint8)
        has_tooth = [_.sum() > 0 for _ in masks]
        labels = labels[has_tooth]
        masks = masks[has_tooth]
        boxes = torchvision.ops.boxes.masks_to_boxes(masks)
        
        # convert teeth number to 1 (ignore the teeth number)
        labels = torch.ones((len(labels), ), dtype=torch.int64)

        # format image and annotation for training
        image = torchvision.tv_tensors.Image(image)
        target = {
            'boxes': torchvision.tv_tensors.BoundingBoxes(boxes, format='XYXY', canvas_size=torchvision.transforms.v2.functional.get_size(image)),
            'masks': torchvision.tv_tensors.Mask(masks),
            'labels': labels,
        }
        image, target = self.transforms(image, target)
        
        return image, target

    def __len__(self):
        return len(self.images)
        
    def __load_datasets(self, root):
        images = []
        masks = []
        for image_fpath in os.listdir(os.path.join(root, 'images')):
            image_fpath = os.path.join(root, 'images', image_fpath)
            if os.path.splitext(image_fpath)[1] == '.jpg':
                image_fname = os.path.basename(image_fpath)
                mask_fpath = os.path.join(root, 'masks', os.path.splitext(image_fname)[0] + '.png')
                if os.path.exists(mask_fpath):
                    images.append(image_fpath)
                    masks.append(mask_fpath)
        return images, masks



通常、モデルの訓練では、データ拡張として画像の拡大縮小、平行移動、回転などを適用することが一般的です。しかし、歯のレントゲン画像の場合、過度なデータ拡張を行うと、本来の画像情報から逸脱し、モデルの学習に悪影響を及ぼす可能性があります。そのため、適切なデータ拡張手法を慎重に選択することが重要です。

### 計算デバイス

PyTorch が GPU を認識できる場合は GPU を利用し、認識できない場合は CPU を利用するように計算デバイスを設定します。

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## モデル構築

Mask R-CNN は高性能なインスタンスセグメンテーションを行うアーキテクチャで、torchvision.models モジュールに含まれています。しかし、torchvision.models で提供されている Mask R-CNN は、COCO データセット向けに設計されており、車や人などの 90 種類の一般的なオブジェクトを対象としています。そのため、そのままでは歯のインスタンスセグメンテーションに適用することができません。

歯のセグメンテーションに対応させるためには、Mask R-CNN の分類モジュールの出力層のユニット数を修正する必要があります。この修正はモデルを呼び出すたびに行う必要があり、手間がかかります。そこで、指定したクラス数に応じてアーキテクチャを生成し、必要に応じて修正を加えられるように、一連の処理を関数として定義します。なお、インスタンスセグメンテーションは物体検出と同様に背景を 1 つのクラスとして扱うため、出力層の数を修正する際には、検出対象のクラス数に 1 を加えた数にする必要があります。

In [ ]:
def maskrcnn(num_classes, weights=None):
    num_classes = num_classes + 1

    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights='DEFAULT')
    
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    model.roi_heads.mask_predictor = torchvision.models.detection.mask_rcnn.MaskRCNNPredictor(in_features_mask, 256, num_classes)
    
    if weights is not None:
        model.load_state_dict(torch.load(weights))
    return model

model = maskrcnn(num_classes=1)
model.to(device)

## モデル訓練

モデルが学習データを効率的に学習できるように、学習アルゴリズム（`optimizer`）、学習率（`lr`）、および学習率を調整するスケジューラ（`lr_scheduler`）を設定します。なお、インスタンスセグメンテーションでは、すべてのピクセルに対して予測結果（分類結果）とラベルの誤差を計算する損失関数を定義する必要がありますが、この損失関数はすでにモデル内で定義されているため、ここで新たに定義する必要はありません。

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

次に、訓練サブセットおよび検証サブセットを読み込み、モデルに入力できる形式に整えます。

In [ ]:
train_loader = torch.utils.data.DataLoader(
                    TeethDataset('teethsegm/train'),
                    batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
valid_loader = torch.utils.data.DataLoader(
                    TeethDataset('teethsegm/valid'),
                    batch_size=4, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

準備が整ったら、訓練を開始します。訓練プロセスでは、訓練と検証を交互に繰り返します。訓練では、訓練データを使ってモデルのパラメータを更新し、その際の損失（誤差）を記録します。検証では、検証データを使ってモデルの予測性能（mAP や IoU）を計算し、その結果を記録します。

In [ ]:
num_epochs = 10
metric_dict = []

for epoch in range(num_epochs):
    # training phase
    model.train()
    epoch_loss_dict = {}

    for images, targets in train_loader:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        batch_loss_dict = model(images, targets)
        batch_tol_loss = 0
        for loss_type, loss_val in batch_loss_dict.items():
            batch_tol_loss += loss_val
            if loss_type in epoch_loss_dict:
                epoch_loss_dict[f'train_{loss_type}'] += loss_val.item()
            else:
                epoch_loss_dict[f'train_{loss_type}'] = loss_val.item()
                
        # update weights
        optimizer.zero_grad()
        batch_tol_loss.backward()
        optimizer.step()
    lr_scheduler.step()


    # validation phase
    model.eval()
    metric = torchmetrics.detection.mean_ap.MeanAveragePrecision()
    iou = 0
    n_targets = 0
    with torch.no_grad():
        for images, targets in valid_loader:
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            pred_targets = model(images)
            metric.update(pred_targets, targets)

            for i in range(len(targets)):
                pred_mask = pred_targets[i]['masks'].squeeze(1).any(dim=0)
                true_mask = targets[i]['masks'].any(dim=0)
                iou += torchmetrics.functional.jaccard_index(pred_mask.unsqueeze(0), true_mask.unsqueeze(0), num_classes=1, task='binary')
                n_targets += 1

    # record training loss
    epoch_loss_dict['train_total_loss'] = sum(epoch_loss_dict.values())
    metric_dict.append({k: v / len(train_loader) for k, v in epoch_loss_dict.items()})
    for k, v in metric.compute().items():
        if k != 'classes':
            metric_dict[-1][k] = v.item()
    metric_dict[-1]['avg_iou'] = iou.item() / n_targets
    metric_dict[-1]['epoch'] = epoch + 1

    print(metric_dict[-1])

訓練データに対する損失と検証データに対する予測性能（mAP）を可視化し、訓練過程を評価します。

In [ ]:
metric_dict = pd.DataFrame(metric_dict)

fig, ax = plt.subplots(1, 2)
ax[0].plot(metric_dict['epoch'], metric_dict['train_total_loss'], label='total')
ax[0].plot(metric_dict['epoch'], metric_dict['train_loss_classifier'], label='classification')
ax[0].plot(metric_dict['epoch'], metric_dict['train_loss_box_reg'], label='bbox')
ax[0].plot(metric_dict['epoch'], metric_dict['train_loss_mask'], label='mask')
ax[0].set_xlabel('epoch')
ax[0].set_ylabel('loss')
ax[0].set_title('Train')
ax[0].legend()
ax[1].plot(metric_dict['epoch'], metric_dict['map_50'], label='mAP(50%)')
ax[1].plot(metric_dict['epoch'], metric_dict['avg_iou'], label='IoU')
ax[1].set_ylim(0, 1)
ax[1].set_xlabel('Epoch')
ax[1].set_ylabel('metrics')
ax[1].set_title('Validation')
ax[1].legend()
plt.tight_layout()
fig.show()

可視化の結果から、エポック数が増えるにつれて訓練データに対する損失が継続的に減少し、5 エポック目から収束し始める傾向がみられました。一方で、検証データに対する検証性能（mAP および IoU）は最初の数エポックですでに高い値に達しいることがわかりました。訓練ではこれで十分と考えらえるので次のステップに進みます。

次に、この手順を他の深層ニューラルネットワークのアーキテクチャ（例えば U-Net など）に適用し、それぞれの検証性能を比較します。この比較により、データセットに最も適したアーキテクチャを選定します。ただし、本節はこの比較を行わずに、Mask R-CNN を最適なアーキテクチャとして扱い、次のステップに進みます。

次のステップでは、訓練サブセットと検証サブセットを統合し、最適と判断したアーキテクチャを最初から訓練します。その準備として、まず訓練サブセットと検証サブセットを結合します。

In [ ]:
!rm -rf teethsegm/trainvalid/images
!rm -rf teethsegm/trainvalid/masks

In [ ]:
!mkdir -p teethsegm/trainvalid/images
!mkdir -p teethsegm/trainvalid/masks

!cp teethsegm/train/images/* teethsegm/trainvalid/images
!cp teethsegm/valid/images/* teethsegm/trainvalid/images
!cp teethsegm/train/masks/* teethsegm/trainvalid/masks
!cp teethsegm/valid/masks/* teethsegm/trainvalid/masks

次に、モデルの構築を行います。先ほど可視化した検証性能の推移グラフを確認した結果、数エポックの訓練で十分に高い予測性能を達成できることがわかりました。そこで、ここでは訓練サブセットと検証サブセットを統合したデータを用いて、5 エポックのみ訓練を行います。

In [ ]:
# model
model = maskrcnn(num_classes=1)
model.to(device)

# training parameters
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, weight_decay=1e-4)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# training data
train_loader = torch.utils.data.DataLoader(
                        TeethDataset('teethsegm/trainvalid'),
                        batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

# training
num_epochs = 5
metric_dict = []
for epoch in range(num_epochs):
    model.train()
    epoch_loss_dict = {}
    for images, targets in train_loader:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        batch_loss_dict = model(images, targets)
        batch_tol_loss = 0
        for loss_type, loss_val in batch_loss_dict.items():
            batch_tol_loss += loss_val
            if loss_type in epoch_loss_dict:
                epoch_loss_dict[f'train_{loss_type}'] += loss_val.item()
            else:
                epoch_loss_dict[f'train_{loss_type}'] = loss_val.item()
        optimizer.zero_grad()
        batch_tol_loss.backward()
        optimizer.step()
    lr_scheduler.step()

    # record training loss
    epoch_loss_dict['train_total_loss'] = sum(epoch_loss_dict.values())
    metric_dict.append({k: v / len(train_loader) for k, v in epoch_loss_dict.items()})
    metric_dict[-1]['epoch'] = epoch + 1
    print(metric_dict[-1])



訓練が完了したら、訓練済みモデルの重みをファイルに保存します。

In [ ]:
model.to('cpu')
torch.save(model.state_dict(), 'teethsegm.pth')

## モデル評価

最適なモデルが得られたら、次にテストデータを用いてモデルを詳細に評価します。

In [ ]:
test_loader = torch.utils.data.DataLoader(
                    TeethDataset('teethsegm/test'),
                    batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

model = maskrcnn(num_classes=1, weights='teethsegm.pth')
model.to(device)
model.eval()

metric = torchmetrics.detection.mean_ap.MeanAveragePrecision()
iou = 0
n_targets = 0
with torch.no_grad():
    for images, targets in test_loader:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        pred_targets = model(images)
        metric.update(pred_targets, targets)
        for i in range(len(targets)):
            pred_mask = pred_targets[i]['masks'].squeeze(1).any(dim=0)
            true_mask = targets[i]['masks'].any(dim=0)
            iou += torchmetrics.functional.jaccard_index(pred_mask.unsqueeze(0), true_mask.unsqueeze(0), num_classes=1, task='binary')
            n_targets += 1


metrics = [{k: v.cpu().detach().numpy().tolist()} for k, v in metric.compute().items()]
metrics.append({'avg_iou': iou.item() / n_targets})
metrics

## 推論

推論時にも、訓練時と同じように torchvision モジュールからアーキテクチャを呼び出し、出力層のクラス数を設定します。その後、`load_state_dict` メソッドを使って、訓練済みの重みファイルをモデルにロードします。これらの操作はすべて `maskrcnn` 関数で定義されているので、その関数を利用します。

In [ ]:
model = maskrcnn(num_classes=1, weights='teethsegm.pth')
model.to(device)
model.eval()

このモデルを利用して推論を行います。まず、1 枚の画像を指定し、PIL モジュールを用いて画像を開き、テンソル形式に変換した後、モデルに入力します。モデルは予測結果としてバウンディングボックスの座標（`bboxes`）、マスク（`masks`）、分類ラベル（`labels`）、および信頼スコア（`scores`）を出力します。ただし、信頼スコアが 0.5 未満のバウンディングボックスは採用せず、信頼スコアが高い結果のみを選択して利用します。

In [ ]:
threshold = 0.5
image_path = 'teethsegm/test/images/13.jpg'

image = PIL.Image.open(image_path).convert('RGB')
input_tensor = torchvision.transforms.v2.functional.to_tensor(image).unsqueeze(0).to(device)
    
with torch.no_grad():
    predictions = model(input_tensor)[0]
    
bboxes = predictions['boxes'][predictions['scores'] > threshold]
masks = predictions['masks'][predictions['scores'] > threshold]
labels = predictions['labels'][predictions['scores'] > threshold]
scores = predictions['scores'][predictions['scores'] > threshold]

検出されたオブジェクトのバウンディングボックスを入力画像に描画します。その後、PIL および matplotlib ライブラリを使用して、画像とその検出結果を可視化します。

In [ ]:
image = PIL.Image.open(image_path).convert('RGBA')
overlay = PIL.Image.new('RGBA', image.size, (255, 255, 255, 0))
overlay_draw = PIL.ImageDraw.Draw(overlay)

for bbox, mask, label, score in zip(bboxes, masks, labels, scores):
    col = tuple([random.randint(0, 255) for _ in range(3)]) + (128,)

    # bbox
    x1, y1, x2, y2 = bbox
    draw = PIL.ImageDraw.Draw(image)
    draw.rectangle(((x1, y1), (x2, y2)), outline=col[:3], width=3)  # BBox is not transparent
    draw.text((x1, y1 - 10), f'{label.item()} ({score:.2f})', fill=col[:3])

    # mask
    mask = mask.squeeze(0).cpu().numpy()
    contours = skimage.measure.find_contours(mask, 0.5)
    for contour in contours:
        contour = np.flip(contour, axis=1).astype(int)
        polygon = [tuple(point) for point in contour]
        overlay_draw.polygon(polygon, fill=col)

blended_image = PIL.Image.alpha_composite(image, overlay)

fig = plt.figure()
ax = fig.add_subplot()
ax.imshow(blended_image)
ax.axis('off')
plt.show()


In [ ]:
!rm -rf teethsegm/trainvalid/images
!rm -rf teethsegm/trainvalid/masks